# Big Data Modeling and Management Assigment - Homework 1

## 🍺 The Beer project  🍺 

As it was shown in classes, graph databases are a natural way of navegating distinct types of data. For this first project we will be taking a graph database to analyse beer and breweries!   

_For reference the dataset used for this project has been extracted from [kaggle](https://www.kaggle.com/ehallmar/beers-breweries-and-beer-reviews), released by Evan Hallmark. Even though the author does not present metada on the origin of the data it is probably a collection of open data from places like [beeradvocate](https://www.beeradvocate.com/)_.

### Problem description

Explore the database via python neo4j connector and/or the graphical tool in the NEO4J webpage. Answer the questions. Submit the results by following the instructions


### Questions

1. How many different countries exist in the database?
1. Most reviews:  
    1. Which `Beer` has the most reviews?  
    1. Which `Brewery` has the most reviews for its beers? [Hint: 5-node path]
    1. Which `Country` has the most reviews for its beers? [Hint: 5-node path]
1. Find the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman?
1. Which Portuguese brewery has the most beers?
1. From those beers (the ones returned from the previous question), which has the most reviews?
1. On average how many different beer styles does each brewery produce?
1. Which brewery produces the strongest beers according to ABV? [Hint: database has NaN values]
1. If I typically enjoy a beer due to its aroma and appearance, which beer style should I try? (Justify your answer well!) [Hint: database has NaN values]
1. Using Graph Algorithms answer **one** of the following questions: [Hint: make sure to clear the graph before using it again]
    1. Which two countries are most similiar when it comes to their **top 10** most produced Beer styles?
    2. Which beer is the most influential when considering beers are conected by users who review them? [Please use limit of 1000 on beer-review-user path]]
    3. Users are connected together by their reviews to beers, taking into consideration the "overall" score they review as a weight, how many communities are formed from these relationships? How many users has the biggest community? [Please use limit of 1000 on beer-review-user path]]
1. Using Graph Algorithms answer **one** of the following questions:
    1. Which beer has the most similar reviews as the beer `Super Bock Stout`? [Hint:inspect two subsets: with and without the beer in question]
    2. Which user is the most influential when it comes to reviews made?
1. If you had to pick 3 beers to recommend using only this database, which would you pick and why? (Justify your answer well!) [Hint: database has NaN values]


Questions 8 to 10 are somewhat open, which means we'll also be evaluating the reasoning behind your answer. So there aren't necessarily bad results there are only wrong criteria, explanations or execution. 
 
### Groups  

Groups should have 4 to 5 people.
You should register your group on **moodle**.

### Submission      

The code used to produce the results and respective explations should be uploaded to moodle. They should have a clear reference to the group, either on the file name or on the document itself. Preferably one Jupyter notebook per group.

Delivery date: Until the **midnight of March 13**

### Evaluation   

This will be 20% of the final grade.   
Each solution will be evaluated on 2 components: correctness of results and simplicity of the solution.  
All code will go through plagiarism automated checks. Groups with the same code will undergo investigation.

## Loading the Database

#### Be sure that you **don't have** the neo4j docker container from the classes running (you can Stop it in the desktop app or with the command "`docker stop Neo4JLab`")


The default container does not have any data whatsoever, we will have to load a database into our docker image:
- Download and unzip the `Neo4JHWData` file provided in Moodle.
- Copy the path of the `Neo4JHWData` folder of the unziped file, e.g. `C:\Users\Utente\Desktop\BDM&M\Neo4JHWData/data`.
- Download and unzip the `Neo4JPlugins` file provided in Moodle.
- Copy the path of the `Neo4JPlugins` folder of the unziped file, e.g. `C:\Users\Utente\Desktop\BDM&M\Neo4JPlugins`.
- Change the code bellow accordingly. As you might have noticed, you do not have a user called `nunoa`, please use the appropriate path that you got from the previous step. Be sure that you have a neo4j docker container running: \

`docker run --name Neo4JHW -p 7474:7474 -p 7687:7687 -d -v "C:\Users\Utente\Desktop\BDM&M\Neo4JPlugins":/plugins -v "C:\Users\Utente\Desktop\BDM&M\Neo4JHWData/data":/data --env NEO4J_AUTH=neo4j/test --env NEO4J_dbms_connector_https_advertised__address="localhost:7473" --env NEO4J_dbms_connector_http_advertised__address="localhost:7474" --env NEO4J_dbms_connector_bolt_advertised__address="localhost:7687" --env NEO4J_dbms_security_procedures_unrestricted=gds.* --env NEO4J_dbms_security_procedures_allowlist=gds.* neo4j:4.4.5`

- Since Neo4j is trying to recognize a new database folder, this might take a bit (let's say 3 minutes), so don't worry.

In [1]:
from neo4j import GraphDatabase
from pprint import pprint

In [2]:
NEO4J_URI="neo4j://localhost:7687"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="test"

In [3]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD), )

### All The Functions you'll need to run queries in Neo4J

In [4]:
def execute_read(driver, query):    
    with driver.session(database="neo4j") as session:
        result = session.execute_read(lambda tx, query: list(tx.run(query)), query)
    return result

In [ ]:
def execute_write(driver, query):
    with driver.session(database="neo4j") as session:
        # Write transactions allow the driver to handle retries and transient errors
        result = session.execute_write(lambda tx, query: list(tx.run(query)), query)
    return result

### Understanding the Database

In [5]:
query = """
        call db.labels();
    """

result = execute_read(driver, query)

pprint(result)

[<Record label='COUNTRIES'>,
 <Record label='CITIES'>,
 <Record label='BREWERIES'>,
 <Record label='BEERS'>,
 <Record label='REVIEWS'>,
 <Record label='STYLE'>,
 <Record label='USER'>]


In [6]:
query = """
        CALL db.relationshipTypes();
    """

result = execute_read(driver, query)

pprint(result)

[<Record relationshipType='REVIEWED'>,
 <Record relationshipType='BREWED'>,
 <Record relationshipType='IN'>,
 <Record relationshipType='HAS_STYLE'>,
 <Record relationshipType='POSTED'>]


## Submission

GROUP NUMBER:

**GROUP 9**

GROUP MEMBERS:

|STUDENT NUMBER|STUDENT NAME|
|---|---|
|20220688|David Castro|
|20220694|Francisco Freitas|
|20220671|Denzel Basso|
|20220556|Vasco Fontoura|

#### 1. How many different countries exist in the database?

In [7]:
#There is 200 different countries in the database
query = """
        MATCH (r:COUNTRIES) 
        RETURN count(distinct r.name) 
    """

result = execute_read(driver, query)

pprint(result)

[<Record count(distinct r.name)=200>]


#### 2. Most reviews:  
    A) Which `Beer` has the most reviews?  
    B) Which `Brewery` has the most reviews for its beers?
    C) Which `Country` has the most reviews for its beers? 

In [8]:
# A)

# Counting the number of relationships between beers and reviews we get the total number of reviews for each beer
# After that, ordering the output by descending order and limiting it to one single line we get the beer with the most reviews

query = """
        MATCH (b:BEERS)-[r:REVIEWED]->(r1:REVIEWS)
        RETURN b.name as Name, count(r) as Reviews
        ORDER BY Reviews DESC
        LIMIT 1
    """

result = execute_read(driver, query)

pprint(result)

[<Record Name='IPA' Reviews=31387>]


In [9]:
# B)

# Firsy we need to connect each brewery with the beers it brews, then to connect the beers to the reviews
# Now we just need to count the number of reviews and get the name of the brewery
# Ordering by descending order and limiting to one output gets us the brewery name with the most reviews to its beers
# and the actual number

query = """
        MATCH (b:BREWERIES)-[:BREWED]->(b2:BEERS)-[r:REVIEWED]->(r2:REVIEWS)  
        RETURN b.name as Brewery,count(r) as Reviews 
        ORDER BY Reviews DESC 
        LIMIT 1
    """

result = execute_read(driver, query)

pprint(result)


[<Record Brewery='Sierra Nevada Brewing Co.' Reviews=175161>]


In [10]:
# C)

# Adding the cities in wich the breweries are located and the countries the corresponding cities are located to the previous
# exercise, we can now check wich country has the most reviews for the beers that are brewed in breweries located in its cities

query = """
       MATCH (c:COUNTRIES)<-[]-(c2:CITIES)<-[:IN]-(b:BREWERIES)-[:BREWED]->(b2:BEERS)-[r:REVIEWED]->(r2:REVIEWS) 
       RETURN c.name as Name,count(r) as Reviews 
       ORDER BY Reviews DESC 
       LIMIT 1
    """

result = execute_read(driver, query)

pprint(result)

[<Record Name='US' Reviews=7675804>]


#### 3. Find the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman?

In [11]:
#The user with the most shared Reviews with the user CTJam is acurtis. 1428 Reviews of the same beers
#Basically we are counting the number of Reviews that each user did (only for the beers that CTJam Reviewed )
query = """
        MATCH (u:USER {name:"CTJman"})<--(r:REVIEWS)<--(b:BEERS)-[c:REVIEWED]->(r1:REVIEWS)-[:POSTED]->(u1:USER)
        RETURN  u1.name, count(c) as Count
        order by Count desc
        LIMIT 1
    """

result = execute_read(driver, query)

pprint(result)

[<Record u1.name='acurtis' Count=1428>,
 <Record u1.name='Texasfan549' Count=1257>]


#### 4. Which Portuguese brewery has the most beers?


In [12]:
# Return the name of the first brewery and the number of beers from the list with maximum beers
# in descending order, where countries = 'PT', connecting Coutries to Cities and Cities to Breweries and Breweries to Beers.

query = f"""
        MATCH (b:BEERS)<-[:BREWED]-(br:BREWERIES)-[:IN]->(c:CITIES)-[:IN]->(co:COUNTRIES)
        WHERE co.name = 'PT'
        RETURN br.name, count(*) AS num_beers
        ORDER BY num_beers DESC
        LIMIT 1
    """

result = execute_read(driver, query)

pprint(result)
#Answer: <Record br.name='Dois Corvos Cervejeira' num_beers=40>

[<Record br.name='Dois Corvos Cervejeira' num_beers=40>]


#### 5. From those beers (the ones produced in the brewery from the previous question), which has the most reviews?


In [13]:
# Return the name of the first beer and the number of reviews from the list of beers 
# from the brewery = 'Dois Corvos Cervejeira', in descending order by reviews, 
#connecting Reviews to Beers and Beers to Breweries.

query = f"""
        MATCH (c:REVIEWS)<-[:REVIEWED]-(b:BEERS)<-[:BREWED]-(br:BREWERIES)
        WHERE br.name = 'Dois Corvos Cervejeira'
        RETURN b.name, count(*) AS num_reviews
        ORDER BY num_reviews DESC
        LIMIT 1
    """

result = execute_read(driver, query)

pprint(result)
#Answer: <Record b.name='Finisterra' num_reviews=10>

[<Record b.name='Finisterra' num_reviews=10>]


#### 6. On average how many different beer styles does each brewery produce?

In [14]:
#SOLUTION1 1
# The average unique stiles per brewery
query = """
        MATCH (br:BREWERIES)-[r:BREWED]->(b:BEERS)-[hs:HAS_STYLE]->(s:STYLE)
        with br.name as brewerie_name, count(distinct s) as Stiles
        RETURN round(avg(Stiles)) as Average_Stiles_per_brewery
    """

result = execute_read(driver, query)

pprint(result)

[<Record Average_Stiles_per_brewery=11.0>]


In [15]:
#SOLUTION 2
# The numbers of uniques stiles per each brewery
query = """
        MATCH (br:BREWERIES)-[r:BREWED]->(b:BEERS)-[hs:HAS_STYLE]->(s:STYLE)
        RETURN br.name as brewerie_name, count(distinct s) as Stiles
        Order by Stiles desc
        LIMIT 100
    """

result = execute_read(driver, query)

pprint(result)

[<Record brewerie_name='Iron Hill Brewery & Restaurant' Stiles=94>,
 <Record brewerie_name='Rock Bottom Restaurant & Brewery' Stiles=93>,
 <Record brewerie_name='Goose Island Beer Co.' Stiles=88>,
 <Record brewerie_name='Sierra Nevada Brewing Co.' Stiles=87>,
 <Record brewerie_name="John Harvard's Brewery & Ale House" Stiles=86>,
 <Record brewerie_name='Cigar City Brewing' Stiles=85>,
 <Record brewerie_name='The Bruery' Stiles=84>,
 <Record brewerie_name='Boston Beer Company (Samuel Adams)' Stiles=84>,
 <Record brewerie_name='Minneapolis Town Hall Brewery' Stiles=81>,
 <Record brewerie_name='Deschutes Brewery' Stiles=81>,
 <Record brewerie_name="Short's Brewing Company" Stiles=81>,
 <Record brewerie_name='Against The Grain Brewery & Smokehouse' Stiles=80>,
 <Record brewerie_name='Harpoon Brewery & Beer Hall' Stiles=80>,
 <Record brewerie_name='Kuhnhenn Brewing Company' Stiles=79>,
 <Record brewerie_name='Boston Beer Works - Canal Street' Stiles=79>,
 <Record brewerie_name='Triumph Brew

#### 7. Which brewery produces the strongest beers according to ABV?

In [16]:
# The strongest beer, with the highest ABV possible of 9.99 is ‘Double Justice’ produced by the brewery 
#‘Unlawful Assembly Brewing Co.’

query = """
        MATCH (br:BREWERIES)-[r:BREWED]->(b:BEERS)
        WHERE b.abv  <> 'nan'
        RETURN b.abv as ABV, br.name as Brewery_name, b.name as Beer_name
        Order by b.abv DESC
        LIMIT 10
    """

result = execute_read(driver, query)

pprint(result)

[<Record ABV='9.99' Brewery_name='Unlawful Assembly Brewing Co.' Beer_name='Double Justice'>,
 <Record ABV='9.99' Brewery_name='Little Spokane Brewing Company' Beer_name='Bull Moose'>,
 <Record ABV='9.99' Brewery_name='Adroit Theory Brewing Company' Beer_name='BLVCK Celebration- Maple Whiskey'>,
 <Record ABV='9.99' Brewery_name='Adroit Theory Brewing Company' Beer_name='BLVCK Celebration- Rum Barrel'>,
 <Record ABV='9.99' Brewery_name='Platt Park Brewing Company' Beer_name="King's Jewels Double IPA">,
 <Record ABV='9.99' Brewery_name='Tanker' Beer_name='Tanker/Mean Sardine Surf Wax'>,
 <Record ABV='9.99' Brewery_name='New Belgium Brewing' Beer_name='Imperial Cascadian'>,
 <Record ABV='9.99' Brewery_name='Brouwerij Bryggja' Beer_name='Mysticum Cosmic Tripel'>,
 <Record ABV='9.99' Brewery_name='Adroit Theory Brewing Company' Beer_name='BLVCK Celebration - Oreo Cookie (Ghost 648)'>,
 <Record ABV='9.99' Brewery_name='Adroit Theory Brewing Company' Beer_name='BLVCK Celebration: Dark Chocola

#### 8. If I typically enjoy a beer due to its aroma and appearance, which beer style should I try?

In [17]:
#We consider that the beer recommended should be the beer with the maximum average of smell and look on the reviews.
# Return the name of the beer style and the average of its´reviews smell and look. For that, we linked the reviews to the 
# beers and the beers to the style. We also converted the attributes smell and look from string to float and consider only
# the attributes not null. In the end, we returned the list by descending order of the attributes of style.

query = f"""
        MATCH (rv:REVIEWS)<-[:REVIEWED]-(b:BEERS)-[r:HAS_STYLE]->(s:STYLE)
        WITH AVG(TOFLOAT(rv.smell)) AS aroma, AVG(TOFLOAT(rv.look)) AS appearance, s
        WHERE aroma IS NOT NULL AND appearance IS NOT NULL
        RETURN s.name AS Name, aroma, appearance
        ORDER BY aroma DESC, appearance DESC
        LIMIT 3
    """

result = execute_read(driver, query)

pprint(result)     
#Answer: <Record Name='New England IPA' aroma=4.413614764761178 appearance=4.383595613210877>

[<Record Name='New England IPA' aroma=4.413614764761178 appearance=4.383595613210877>,
 <Record Name='American Imperial Stout' aroma=4.262885690516247 appearance=4.286394107776094>,
 <Record Name='Belgian Gueuze' aroma=4.230876612146052 appearance=4.17496417453178>]


#### 9. Using Graph Algorithms answer **one** of the following questions:
    1. Which two countries are most similiar when it comes to their **top 10** most produced Beer styles?
    2. Which beer is the most influential when considering beers are conected by users who review them?
    3. Users are connected together by their reviews to beers, taking into consideration the "overall" score they review as a weight, how many communities are formed from these relationships? How many users has the biggest community?

In [18]:
# Question 9.1 "Which two countries are most similiar when it comes to their **top 10** most produced Beer styles?"

# To find out which 2 countries are most similar when it comes to their "top 10" most produced Beer Styles, we first need to
# find the top 10 most produced Beer styles for each country.
# To do that, we first checked how many different Beers each country produce and the respective Beer Style for each one of 
# Beers. To find out the top 10, since we already have the Beer Styles that each country produce we just need to count how 
# many times each Style appears. The 10 Beer Styles with hihger frequency are considered the "top 10" most produced.

# For this type of comparison, we did not consider the countries that produce less than 10 different Beer Styles. 
# So countries with per example just 9 types of Beer Style were not considered.
# We put this condition to avoid cases where 2 countries only with 1 type of Beer Style produced of having a similiraty of 1.

# Step 1 - Create an appropriate graph
try:
    query = """
        CALL gds.graph.project.cypher(
            'example_1',
      'MATCH (n) WHERE n:COUNTRIES OR n:STYLE RETURN id(n) AS id, labels(n) AS labels',
      '        MATCH (c:COUNTRIES)<-[]-(CITIES)<-[]-(BREWERIES)-[]->(BEERS)-[]->(s:STYLE)
        WITH c, count (distinct s.name) as nomes
        WHERE nomes>9
        MATCH (c:COUNTRIES)<-[]-(CITIES)<-[]-(BREWERIES)-[]->(BEERS)-[]->(s:STYLE)
        WITH c,s,count(s.name) as count, count (distinct s.name) as sitios
        ORDER BY c,count DESC
        WITH COLLECT (id(s)) as nomes,c
        WITH id(c) as source,  nomes[..10] as target_1
        UNWIND target_1 as target        
        RETURN  source,target') 

        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

ClientError("Failed to invoke procedure `gds.graph.project.cypher`: Caused by: java.lang.IllegalArgumentException: A graph with name 'example_1' already exists.")


In [19]:
# Step 2 - Run the algorithm

# Since we want the most similar Countries we have to order by similarity (with the highest on top)
# Based on th results we can say that the 2 most similar Countries in terms of their "top 10" most Beer Styles produced
# are 'AU (Australia)' and 'NZ (New Zealand)' with similarity of 1.

try:
    query = """
        CALL gds.nodeSimilarity.stream('example_1',{})
            YIELD node1, node2, similarity
            with gds.util.asNode(node1).name AS User1, gds.util.asNode(node2).name AS User2, similarity
            RETURN User1, User2, similarity
            ORDER BY similarity DESCENDING 
            limit 1
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

[<Record User1='AU' User2='NZ' similarity=1.0>]


#### 10. Using Graph Algorithms answer **one** of the following questions:
    1. Which beer has the most similar reviews as the beer `Super Bock Stout`?
    2. Which user is the most influential when it comes to reviews made?

In [21]:
# 10.1 - First we try to apply the Node Similarity algorithm but it gave us always problems with the available space to run it.
# We even try to limit our queries however it could decrease its accuracy. So because of this we manage to answer this question
# using another graph algorithm.

# We will reply to 10.1 returning the beer most similar to `Super Bock Stout` based on the average of the 'overall' attribute 
# from REVIEWS' nodes connected to each beer, using Graph Algorithms.

# Create a property in the BEERS nodes with the average of the attribute "overall" of all REVIEWS nodes connected to that beer.
# Convert to Float the String "overall" attribute on REVIEWS.
query = f"""
        MATCH (b:BEERS)-[:REVIEWED]->(r:REVIEWS)
        WITH b, avg(toFloat(r.overall)) AS avgOverall     
        SET b.avgOverall = avgOverall
    """

result = execute_write(driver, query)

pprint(result)   

[]


In [22]:
# For curiosity purposes only, check the average Overall of all the reviews connected to Super Bock Stout beer.
try:
    query = """
            MATCH (b:BEERS {name: 'Super Bock Stout'})
            RETURN b.avgOverall
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

[<Record b.avgOverall=3.061320754716981>]


In [23]:
# Create an appropriate graph called 'KNN' with only the node 'BEERS' and the property 'avgOverall' just created.

try:
    query = """
            CALL gds.graph.project(
                'KNN',
                {
                    BEERS: {
                        properties: ['avgOverall']
                    }
                },
                '*'
            );
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)


ClientError("Failed to invoke procedure `gds.graph.project`: Caused by: java.lang.IllegalArgumentException: A graph with name 'KNN' already exists.")


In [24]:
# From the cell above we can see that the graph is considering 358.873 nodes.
# So, now let´s count how many BEERS' nodes there are in our dataset, to ensure that we will compare 'Super Bock Stout' with
#all beers
query = f"""
        MATCH (b:BEERS)
        RETURN COUNT(b)
    """

result = execute_write(driver, query)

pprint(result)  

# 358873 = 358873

[<Record COUNT(b)=358873>]


In [25]:
# Now using the graph created, we apply the KNN algorithm returning only the 10th nodes most similar (topK:10) based on the 
# attribute 'avgOverall'. We set a random Seed to control the randomness of the algorithm, which implies to set concurrency to 1.
# We set Sample rate to = 1 to include all number of comparisons per node. And set deltaThreshold to 0 to don't stop the algorithm earlier.

#Then we yield the first node as 'Super Bock Stout' and the list of beers most similar to this one as well as the similarity
#score, by descending order and answer is the first beer that appears.
try:
    query = """
            CALL gds.knn.stream('KNN', {
                topK: 10,
                nodeProperties: ['avgOverall'],
                randomSeed: 1337,
                concurrency: 1,
                sampleRate: 1.0,
                deltaThreshold: 0.0
            })
            YIELD node1, node2, similarity
            WHERE gds.util.asNode(node1).name = 'Super Bock Stout'
            RETURN gds.util.asNode(node1).name AS Person1, gds.util.asNode(node2).name AS Person2, similarity
            ORDER BY similarity DESCENDING, Person1, Person2
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)
    
#Answer :'Soproni'  with similarity = 0.9994903160040784>

[<Record Person1='Super Bock Stout' Person2='Soproni' similarity=0.9994903160040784>,
 <Record Person1='Super Bock Stout' Person2='Bavaria NA' similarity=0.9988221436984692>,
 <Record Person1='Super Bock Stout' Person2='Ellis Island Hefe Weiss' similarity=0.9988221436984692>,
 <Record Person1='Super Bock Stout' Person2='Pivotal Porter (Barrel Aged)' similarity=0.9988221436984692>,
 <Record Person1='Super Bock Stout' Person2="'naBIO" similarity=0.9988221436984688>,
 <Record Person1='Super Bock Stout' Person2="Esser's Anniversary Ale" similarity=0.9988221436984688>,
 <Record Person1='Super Bock Stout' Person2="Harrington's East Indies Lager" similarity=0.9988221436984688>,
 <Record Person1='Super Bock Stout' Person2='Project One Ale' similarity=0.9988221436984688>,
 <Record Person1='Super Bock Stout' Person2='Soulless Black Saison' similarity=0.9988221436984688>,
 <Record Person1='Super Bock Stout' Person2='Stoute Liefde' similarity=0.9988221436984688>]


#### 11. If you had to pick 3 beers to recommend using only this database, which would you pick and why?

### To answer this qustion we proposed 4 different way to recommend a beer:
##### 1st the most popular beer (with more reviews): 'IPA'
    Note_1: popular doesn't mean good
##### 2nd the beer with highest average reviews: 'O.W.K.'
    Note_2: the beers with 5 stars thend to have few reviews
##### 3rd the most reviewd beer with at least 4 as average review: 'Imperial Stout'
    Note_3: The beer is relatively good and well known
##### 4th The beer with higest average reviews and at least 100 reviews: 'Good Morning'
    Note_4: This is the best beer with at least 100 reviews to confirm it

In [27]:
#SOLUTION 1
#Beer with the most reviews
query = """
        MATCH (b:BEERS)-[r:REVIEWED]->(re:REVIEWS)
        WHERE re.overall  <> 'nan' AND toFloat(re.overall) > 0
        RETURN b.name as beer_name, count(r) as Reviews, AVG(toFloat(re.overall)) as Avg_review
        Order by Reviews desc, Avg_review desc
        LIMIT 3
    """

result = execute_read(driver, query)

pprint(result)

[<Record beer_name='IPA' Reviews=16527 Avg_review=3.991090337024293>,
 <Record beer_name='Porter' Reviews=10900 Avg_review=3.991720183486247>,
 <Record beer_name='Imperial Stout' Reviews=10176 Avg_review=4.133451257861628>]


In [28]:
#SOLUTION 2
#Beers with the highest average reviews
query = """
        MATCH (b:BEERS)-[r:REVIEWED]->(re:REVIEWS)
        WHERE re.overall  <> 'nan' AND toFloat(re.overall) > 0
        WITH b.name as beer_name, round(AVG(toFloat(re.overall)),1) as Avg_review, Count(r) as Reviews
        RETURN beer_name, Avg_review, Reviews
        Order by Avg_review desc, Reviews desc
        LIMIT 3
    """

result = execute_read(driver, query)

pprint(result)

[<Record beer_name='O.W.K.' Avg_review=5.0 Reviews=7>,
 <Record beer_name='Big Hefe' Avg_review=5.0 Reviews=5>,
 <Record beer_name='Batch Two' Avg_review=5.0 Reviews=4>]


In [29]:
#SOLUTION 3
#Beers with the most reviews and an average of 4 or more
query = """
        MATCH (b:BEERS)-[r:REVIEWED]->(re:REVIEWS)
        WHERE re.overall  <> 'nan' AND toFloat(re.overall) > 0
        WITH b.name as beer_name, count(r) as Reviews, AVG(toFloat(re.overall)) as Avg_review
        WHERE Avg_review >=4
        RETURN beer_name, round(Avg_review,1), Reviews
        Order by Reviews desc, Avg_review desc
        LIMIT 3
    """

result = execute_read(driver, query)

pprint(result)

[<Record beer_name='Imperial Stout' round(Avg_review,1)=4.1 Reviews=10176>,
 <Record beer_name='Breakfast Stout' round(Avg_review,1)=4.4 Reviews=7663>,
 <Record beer_name='KBS (Kentucky Breakfast Stout)' round(Avg_review,1)=4.5 Reviews=7042>]


In [30]:
# #Undesrtand the average number of reviews per beer
# query = """
#         MATCH (b:BEERS)-[r:REVIEWED]->(re:REVIEWS)
#         WHERE re.overall  <> 'nan'AND toFloat(re.overall) > 0
#         WITH b.name as beer_name, count(r) as Reviews, AVG(toFloat(re.overall)) as Avg_review
#         RETURN round(avg(Avg_review)) as Average_reviews_per_beer
#         LIMIT 3
#     """

# result = execute_read(driver, query)

# pprint(result)

# #The average is 4, for solution 4 we will choose at least 100 reviews due to the lowest average per beer
#     #(we want to have a beer with enouth reviews)

[<Record Average_reviews_per_beer=4.0>]


In [31]:
#SOLUTION 4
#Beers with the highest average reviews with at least 100 reviews
query = """
        MATCH (b:BEERS)-[r:REVIEWED]->(re:REVIEWS)
        WHERE re.overall  <> 'nan' AND toFloat(re.overall) > 0
        WITH b.name as beer_name, round(AVG(toFloat(re.overall)),1) as Avg_review, Count(r) as Reviews
        WHERE Reviews >= 100
        RETURN beer_name, Avg_review, Reviews
        Order by Avg_review desc, Reviews desc
        LIMIT 3
    """

result = execute_read(driver, query)

pprint(result)

[<Record beer_name='Good Morning' Avg_review=4.8 Reviews=918>,
 <Record beer_name='Kentucky Brunch Brand Stout' Avg_review=4.8 Reviews=434>,
 <Record beer_name='R&D Sour Fruit (Very Sour Blackberry)' Avg_review=4.8 Reviews=114>]
